# Detecting Objects using the Wallaroo mobilenet Pipeline in a Video Stream

In this tutorial we will explore using the mobilenet pipeline we created in step 3 to run inference on the frames in a Video and then draw the identified object's bounding boxes, classification and classification confidence.

In [1]:
import torch
import pickle
import wallaroo
import os
import numpy as np
import json
import requests
import time
import imutils

from CVDemoUtils import CVDemo



/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wl = wallaroo.Client()

In [9]:
ws = wl.list_workspaces()
for w in ws:
    if w.name() == 'computer-vision':
        wl.set_current_workspace(w)

In [10]:
model_name = "mobilenet"

In [11]:
mobilenet_model = wl.upload_model(model_name, "models/mobilenet.pt.onnx")

In [12]:
#wl.upload_model('post-process', "post-process.py")

In [13]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("8Gi").build()

In [14]:
#p = p.add_validation('price too high', housing_model.outputs[0][0] < 100.0)
pipeline = wl.build_pipeline('mobilenet-pp') \
            .add_model_step(mobilenet_model) \
            .deploy(deployment_config = deployment_config)


Waiting for deployment - this will take up to 45s ...........................................

WaitForDeployError: Cannot deploy pipeline due to insufficient resources. Your pipeline needs 8Gi of memory to run but there is not enough memory currently available. Please try again or un-deploy pipelines not in use to adjust the resources that are available for your Wallaroo instance. Contact your Wallaroo platform administrator for additional support.

In [ ]:
url = pipeline._deployment._url()
print(url)


# Building, configuring, and scheduling Assays

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import wallaroo
from wallaroo.assay_config import BinMode, Aggregation, Metric


plt.rcParams["figure.figsize"] = (12,6)
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
client = wallaroo.Client()

pipeline_name = 'mypipeline'

pipeline = client.pipelines_by_name("mobilenet-pp")[0]

In [ ]:
import datetime
baseline_start = datetime.datetime.fromisoformat('2022-10-31T00:00:00+00:00')
baseline_end = datetime.datetime.fromisoformat('2022-11-01T00:00:00+00:00')
last_day = datetime.datetime.fromisoformat('2022-11-30T00:00:00+00:00')

In [ ]:
#assay_name = "mobilenet-assay"
#assay_builder = client.build_assay(assay_name, pipeline, model_name, baseline_start, baseline_end)


In [ ]:
#baseline_run = assay_builder.build().interactive_baseline_run()
#baseline_run.baseline_stats()


## Test the pipeline in a video stream

### Initialize some Vars

Initialize the COCO Classes, meaning the classificaitons found on the images and the default width and height all images are resized to

In [ ]:

from CVDemoUtils import CVDemo

# The size the image will be resized to
width = 640
height = 480

# Only objects that have a confidence > confidence_target will be displayed on the image
confidence_target = 0.75

cvDemo = CVDemo()


### Detect and Classify Objects in Video using Wallaroo Shadow Deployment

Next we will load each frame in the input-video feedubg ut to the pipeline for inferencing.  Then using the results we will draw a bounding box around each identified object, print its classification, and the model's confidence that the prediction is accurate on the frame and save the frame to an output video.

As we are executing this notice the time it takes to process each frame.  In the next section we will discuss ways to improve this performance.

In [15]:
# The size the image will be resized to
width = 640
height = 480

input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-inferenced.mp4"


red = (0, 0, 255)
config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : 15, # Frames per second
    'endpoint-url' : url, # the pipelines rest api endpoint url
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    'max-frames' : 400, # the # of frames to capture in the output video
    'skip-frames' : 75, # the # of frames to capture in the output video
    'confidence-target' : 0.90, # only display bounding boxes with confidence > provided #
    'color':red,
    'blur-frame-start':130,
    'blur-frame-end':175
}
cvDemo.DEBUG = True
cvDemo.simulateDriftWhileDetectingAndClassifyingObjectsInVideoUsingPipeline(config)
print("We are done.")



NameError: name 'url' is not defined

In [ ]:
#pipeline.undeploy()
#for d in wl.list_deployments():
#    d.undeploy()
    

### Conclusion

Notice how simple it is to take the mobilenet object detectors rest api endpoint url and use it in a video stream.  Now its your turn.  Upload a video to this notebook and replace the input-video with the path of the uploaded video.  Update the output video accordingly.

See how well it works for you.  Good luck!


In [ ]:
import cv2    
input_video = "videos/amazon-fresh-go.mp4"
cap = cv2.VideoCapture(input_video)
  
output = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*'MPEG'), 25, (1280,720))

try:
    cnt=0
    while(True):
        ret, frame = cap.read()
        if(ret):
            # adding rectangle on each frame
            #cv2.rectangle(frame, (100, 100), (500, 500), (0, 255, 0), 3)

            # writing the new frame in output
            print(frame.shape)
            print("frame "+str(cnt))

            output.write(frame)
        cnt += 1
        if cnt == 200:
            break;
except KeyboardInterrupt:
    print("Exiting")
    
output.release()
cap.release()

In [ ]:
video_path = "videos/amazon-fresh-go.mp4"
size = images[0].shape[1], images[0].shape[0] ## <<<--- NOTICE THIS
video = cv2.VideoWriter(video_path,cv2.VideoWriter_fourcc(*'DIVX'), 60, size)  

for img in images:  
    video.write(img)

cv2.destroyAllWindows()
video.close()
